In [6]:
from agential.cog.agent.react import ReActAgent
from langchain.chat_models.openai import ChatOpenAI



In [7]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=api)

test = ReActAgent(llm=llm, max_steps=6)

In [8]:
from agential.cog.prompts.react import REACT_AMBIGQA_EXAMPLES

output = test.generate(question = "When did the simpsons first air on television?", examples= REACT_AMBIGQA_EXAMPLES )

/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


# Example of AmbigQA

In [9]:
print(REACT_AMBIGQA_EXAMPLES)


Question: When did men's figure skating become a summer Olympic sport? 
Thought 1: Let's search the question in google 
Action 1: Search[When did men's figure skating become a summer Olympic sport?] 
Observation 1: [Figure skating at the Olympic Games - Wikipedia] Figure skating was first contested in the Olympic Games at the 1908 Summer Olympics . Since 1924, the sport has been a part of the Winter Olympic Games . 
Thought 2: The evidence suggests Figure skating became an Olympic sport at the 1908 Summer Olympics, and has been a part of the Winter Olympic Games in 1924. 
Action 2: Finish[1908]

Question: When was the last time the USA men's national soccer team missed the World Cup? 
Thought1: Let's search the question in google Action 1: Search[When was the last time the USA men's national soccer team missed the World Cup? site: wikipedia.org] 
Observation 1: [United States at the FIFA World Cup - Wikipedia] The United States participated in every World Cup from 1990 through 2014, b

In [10]:
output

[ReActOutput(thought="Thought: Let's search for the air date of The Simpsons on television.", action='Action: Search[When did The Simpsons first air on television?]', observation='Observation 1: If I Did It: Confessions of the Killer is a book by O.J. Simpson and Pablo Fenjves, in which Simpson puts forth a hypothetical description of the murders of Nicole Brown Simpson and Ron Goldman. According to Fenjves, the book is based on extensive discussions with Simpson, while Simpson\'s former manager, Norman Pardo, claimed that Simpson was not involved in writing the book but rather accepted $600,000 to say he had written it and to conduct an interview.Simpson was acquitted of the murders in a criminal trial (People v. Simpson) but later was found financially liable in a civil trial. Although the original release of the book was canceled shortly after it was announced in November 2006, 400,000 physical copies of the original book were printed, and copies of it had been leaked online by June

# OUTPUT

In [11]:
for i in output:
    print(i.thought)
    
    print(i.action)
    print(i.observation)

Thought: Let's search for the air date of The Simpsons on television.
Action: Search[When did The Simpsons first air on television?]
Observation 1: If I Did It: Confessions of the Killer is a book by O.J. Simpson and Pablo Fenjves, in which Simpson puts forth a hypothetical description of the murders of Nicole Brown Simpson and Ron Goldman. According to Fenjves, the book is based on extensive discussions with Simpson, while Simpson's former manager, Norman Pardo, claimed that Simpson was not involved in writing the book but rather accepted $600,000 to say he had written it and to conduct an interview.Simpson was acquitted of the murders in a criminal trial (People v. Simpson) but later was found financially liable in a civil trial. Although the original release of the book was canceled shortly after it was announced in November 2006, 400,000 physical copies of the original book were printed, and copies of it had been leaked online by June 2007.The book was originally due to be publishe